In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False 
cache_dir = "/data1/debajyoti/test/llms/models"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Telugu-LLM-Labs/Indic-gemma-7b-finetuned-sft-Navarasa-2.0",
    cache_dir = cache_dir,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map="auto"
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

input_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

input_text = input_prompt.format(
        """You are a sarcasm recognition assistant judging if an 'Input' is sarcastic or not. Dont give output anything else.
    If the 'Input' is sarcastic, you need to output your final 'Output' as 'sarcasm'. 
    If the 'Input' is non-sarcastic, you need to output your final 'Output' as 'Non-sarcasm'.""", # instruction
        "Chalo protocol pata chal gaya. Vice President agar Muslim na hote toh shayad kabhi pata hi nahi chaltapic.twitter.com/bOLAEk0g4E", # input
        "", # output - leave this blank for generation!
    )

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
response = tokenizer.batch_decode(outputs)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/data1/debajyoti/test/llms/envllm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


==((====))==  Unsloth 2024.11.5: Fast Gemma patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.154 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.
Loading checkpoint shards: 100%|██████████| 8/8 [00:07<00:00,  1.13it/s]
Unsloth: Will load Telugu-LLM-Labs/Indic-gemma-7b-finetuned-sft-Navarasa-2.0 as a legacy tokenizer.


In [2]:
response

["<bos>\n### Instruction:\nYou are a sarcasm recognition assistant judging if an 'Input' is sarcastic or not. Dont give output anything else.\n    If the 'Input' is sarcastic, you need to output your final 'Output' as 'sarcasm'. \n    If the 'Input' is non-sarcastic, you need to output your final 'Output' as 'Non-sarcasm'.\n\n### Input:\nChalo protocol pata chal gaya. Vice President agar Muslim na hote toh shayad kabhi pata hi nahi chaltapic.twitter.com/bOLAEk0g4E\n\n### Response:\nNon-sarcasm<eos>"]

In [3]:
# input_text = input_prompt.format(
#         """You are a sarcasm recognition assistant judging if an 'Input' is sarcastic or not. Dont give output anything else.
#     If the 'Input' is sarcastic, you need to output your final 'Output' as 'Sarcasm'. 
#     If the 'Input' is non-sarcastic, you need to output your final 'Output' as 'Non-sarcasm'.""", # instruction
#         "What happens if Usain Bolt misses his bus? He waits for it at the next stop.", # input
#         "", # output - leave this blank for generation!
#     )

# inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
# response = tokenizer.batch_decode(outputs)
# response

In [4]:
import pandas as pd
from tqdm import tqdm
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
train_df_eng = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/1Sarcasm_English.csv")
train_df_eng = train_df_eng[:2000]
train_df_hin = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/New_Sarcasm_Hindi.csv")
train_df_hin = train_df_hin[:2000]
train_df = pd.concat([train_df_eng, train_df_hin], ignore_index=True)
# train_df = train_df_hin
# train_df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/1Sarcasm_Codemix.csv")
val_df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/splits/sarcasm/val.csv")
test_df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/splits/sarcasm/test.csv")

test_df

,Sentence,Tag
0,"Mera bhanja mujhe ""papa ka sala"" bulata hai......",0
1,Tu sale TRIPLE TALAQ nd HALALA Ke baad ki sant...,0
2,aur aap a0ni politics karne me vyast hai.. #Bi...,0
3,Theek hai waha at least Janki ma ka mandir hai...,0
4,Sir mujhe cricket acdemy join karna hai mai ka...,0
...,...,...
520,aaj bjp aur chadhi gang aise khus ho rahe hai ...,0
521,@SGanguly99 sir aapnio jodi aamar saath na den...,0
522,#Irony: Pran ka pran chala gaya.. RIP Sher Khan!,1
523,Ye bhi mat bhulo ki triple talaq ko ban lagane...,0


In [11]:
test_df['Tag'].value_counts()

Tag
0    475
1     50
Name: count, dtype: int64

In [111]:
df = pd.read_csv("/data1/debajyoti/code-mix-humor-sarcasm-detection/outputs/sarcasm/few_shots/iacv2_hin_eng_llama.csv")
df

,Sentence,Tag
0,और इसके बजाय खानाबदोश चरवाहों की पौराणिक कथाओं...,Non-sarcasm
1,आप जानते हैं कि अधिकांश सार्वजनिक हेल्थकेयर सि...,Non-sarcasm
2,गलत- मुझे लगता है कि ऑलसेन उन लोगों का हवाला द...,Non-sarcasm
3,"Well that may be your contention, but medicall...",Non-sarcasm


In [112]:
# Convert dataframe to specified format
formatted_few_shot_examples = []
for _, row in df.iterrows():
    formatted_few_shot_examples.append({'role': 'user', 'content': row['Sentence']})
    formatted_few_shot_examples.append({'role': 'assistant', 'content': row['Tag']})
    
formatted_few_shot_examples

[{'role': 'user',
  'content': 'और इसके बजाय खानाबदोश चरवाहों की पौराणिक कथाओं में मेरा विश्वास रखें?यह कुछ जानने के लिए बहुत अधिक तर्कसंगत है, लेकिन अनजाने जादू है, जिसने सब कुछ अस्तित्व में रखा और फिर सभी सबूतों को बनाया, ताकि वह ऐसा हो जैसे उसने ऐसा नहीं किया।'},
 {'role': 'assistant', 'content': 'Non-sarcasm'},
 {'role': 'user',
  'content': 'आप जानते हैं कि अधिकांश सार्वजनिक हेल्थकेयर सिस्टम के तहत आप अभी भी निजी देखभाल खरीद सकते हैं यदि आप वास्तव में चाहते हैं, तो आप नहीं हैं? जैसा कि ओल्डसर्ज के बीन पुशर्स के दावे के लिए यह तय करना है कि आपको क्या देखभाल मिलती है, उनके लिए दूसरा नाम नहीं है "दावे समायोजक"?जब मैं एक डॉक्टर को देखने जाता हूं तो वह वही होता है जो मेरा इलाज तय करता है, किसी और को नहीं।उसे फंडिंग की जांच करने के लिए किसी को भी कॉल करने की ज़रूरत नहीं है, अगर यह किसी के लिए भी कवर किया गया है, तो यह मेरे लिए भी कवर किया गया है।'},
 {'role': 'assistant', 'content': 'Non-sarcasm'},
 {'role': 'user',
  'content': 'गलत- मुझे लगता है कि ऑलसेन उन लोगों का हवाला देते हुए बहु

In [113]:
formatted_few_shot_examples = ""
for index, row in df.iterrows():
    if index == 0 or index == 3:
        formatted_few_shot_examples += f"""'Input': "{row['Sentence']}"\n'Response':"{row['Tag']}"\n\n"""
    # formatted_few_shot_examples += f"""'Input': "{row['Sentence']}"\n'Response':"{row['Tag']}"\n\n"""
    # if index == 1:
    #     break

# Display the result
print(formatted_few_shot_examples)

'Input': "और इसके बजाय खानाबदोश चरवाहों की पौराणिक कथाओं में मेरा विश्वास रखें?यह कुछ जानने के लिए बहुत अधिक तर्कसंगत है, लेकिन अनजाने जादू है, जिसने सब कुछ अस्तित्व में रखा और फिर सभी सबूतों को बनाया, ताकि वह ऐसा हो जैसे उसने ऐसा नहीं किया।"
'Response':"Non-sarcasm"

'Input': "Well that may be your contention, but medically that is not necessarily true. Maybe being gay doesn't mean you are suicidal or depressed, but there is more of it going on in the gay life than in the hetero. That in itself is a wake up call. Depression and suicide are harmful, even if it doesn't touch you personally. If it were as simple as you say the APA wouldn't be re-evaluating "reorientation therapies" would they and with positive critiques too. You wouldn't have prominent gay activists like Camile Paglia deriding the gay community for being malcontent when those unhappy with being gay seek and desire the way to be hetero."
'Response':"Non-sarcasm"




: 

### Create prompt, parse response and  generate response

In [102]:
from transformers import pipeline
import re
import json

# Define the function to create the prompt
def create_prompt(input, num_examples):

    # System prompt 
    system_prompt = """You are a sarcasm recognition assistant judging if an 'Input' is sarcastic or not.
        If the 'Input' is sarcastic, you need to give your final 'Output' as "Sarcasm". 
        If the 'Input' is non-sarcastic, you need to give your final 'Output' as "Non-sarcasm". Dont output anything else. """

    messages = [system_prompt + formatted_few_shot_examples, input]
    # messages = [system_prompt, input]

    return messages

def parse_response(response):
    """
    Extracts the classification label and explanation from the model-generated response,
    starting from the line containing the final "Answer:".

    Args:
        response (str): The response generated by the model.

    Returns:
        tuple: A tuple containing the classification label ("Sarcasm" or "Non-sarcasm") 
               and the explanation text.
    """
    # print(response)
    if len(response) > 0:        
        if "Non-sarcasm" in response:
            return "0"
        elif "Sarcasm" in response:
            return "1"
        else:# If no label is found, return None
            return None

def generate_responses(prompt):
    
    input_prompt = """
    ### Instruction:
    {}

    ### Input:
    {}

    ### Response:
    {}"""

    input_text = input_prompt.format(
            f"""{prompt[0]}""", # instruction
            f"{prompt[1]}", # input
            "", # output - leave this blank for generation!
        )

    inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 400, use_cache = True)
    response = tokenizer.batch_decode(outputs)
    
    return response

In [103]:
def process_data(data_filepath, output_filepath):
    # df = pd.read_csv(data_filepath)
    df = test_df
    # df = df[:5]
    
    
    results = []
    num_examples = 15 # number of shots in few-shot
    # Loop over the DataFrame and pass the text to generate response
    for index, row in tqdm(list(df.iterrows())):
        prompt = create_prompt(row['Sentence'], num_examples)
        model_response = generate_responses(prompt)
        # print(model_response)
        generated_label = parse_response(model_response)

        # veracity, explanation = parse_response(model_response)
        
        results.append({
            'Sentence': row['Sentence'],
            'Label': row['Tag'],
            'Response': model_response,
            'Gen_label': generated_label,
            # 'Explanation': explanation
        })
        # break
    
    results = pd.DataFrame(results)
    
    return results

# Example file paths, replace with your actual paths
data_filepath = '/data1/debajyoti/code-mix-humor-sarcasm-detection/Dataset/splits/humor/test.csv'
output_filepath = '/data1/debajyoti/test/llms/data/humor'

# Run the data processing
result = process_data(data_filepath, output_filepath)

  0%|          | 0/525 [00:00<?, ?it/s]

100%|██████████| 525/525 [02:41<00:00,  3.25it/s]


In [104]:
result

,Sentence,Label,Response,Gen_label
0,"Mera bhanja mujhe ""papa ka sala"" bulata hai......",0,[<bos>\n ### Instruction:\n You are a sa...,None
1,Tu sale TRIPLE TALAQ nd HALALA Ke baad ki sant...,0,[<bos>\n ### Instruction:\n You are a sa...,None
2,aur aap a0ni politics karne me vyast hai.. #Bi...,0,[<bos>\n ### Instruction:\n You are a sa...,None
3,Theek hai waha at least Janki ma ka mandir hai...,0,[<bos>\n ### Instruction:\n You are a sa...,None
4,Sir mujhe cricket acdemy join karna hai mai ka...,0,[<bos>\n ### Instruction:\n You are a sa...,None
...,...,...,...,...
520,aaj bjp aur chadhi gang aise khus ho rahe hai ...,0,[<bos>\n ### Instruction:\n You are a sa...,None
521,@SGanguly99 sir aapnio jodi aamar saath na den...,0,[<bos>\n ### Instruction:\n You are a sa...,None
522,#Irony: Pran ka pran chala gaya.. RIP Sher Khan!,1,[<bos>\n ### Instruction:\n You are a sa...,None
523,Ye bhi mat bhulo ki triple talaq ko ban lagane...,0,[<bos>\n ### Instruction:\n You are a sa...,None


In [105]:
from pydantic import BaseModel, validator
import re

class ModelResponse(BaseModel):
    instruction: str
    input_text: str
    output_label: str

    # @validator('instruction', pre=True, always=True)
    # def extract_instruction(cls, value, values):
    #     # Extract the Instruction part
    #     match = re.search(r'### Instruction:\n(.+?)\n\n### Input:', value, re.DOTALL)
    #     return match.group(1).strip() if match else ""

    # @validator('input_text', pre=True, always=True)
    # def extract_input_text(cls, value, values):
    #     # Extract the Input part
    #     match = re.search(r'### Input:\n(.+?)\n\n### Response:', value, re.DOTALL)
    #     return match.group(1).strip() if match else ""

    @validator('output_label', pre=True, always=True)
    def extract_output_label(cls, value, values):
        # Extract the Response part
        match = re.search(r'### Response:\n(.+?)<eos>', value, re.DOTALL)
        return match.group(1).strip() if match else ""

# Example usage
response_text = (
    "<bos>\n### Instruction:\nYou are a sarcasm recognition assistant judging if an 'Input' "
    "is sarcastic or not. Dont give output anything else.\n    If the 'Input' is sarcastic, "
    "you need to output your final 'Output' as 'Sarcasm'. \n    If the 'Input' is non-sarcastic, "
    "you need to output your final 'Output' as 'Non-sarcasm'.\n\n### Input:\nWhat happens if Usain Bolt "
    "misses his bus? He waits for it at the next stop.\n\n### Response:\nHumor<eos>"
)

# Create ModelResponse object
parsed_response = ModelResponse(instruction=response_text, input_text=response_text, output_label=response_text)

# Access parsed fields
# print("Instruction:", parsed_response.instruction)
# print("Input Text:", parsed_response.input_text)
print("Output Label:", parsed_response.output_label)

Output Label: Humor


/tmp/ipykernel_1148884/2053982226.py:21: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.9/migration/
  @validator('output_label', pre=True, always=True)


In [ ]:
for index, row in tqdm(list(result.iterrows())):
    parsed_response = ModelResponse(instruction=row['Response'][0], input_text=row['Response'][0], output_label=row['Response'][0])

    # Access parsed fields
    print("Output Label:", parsed_response.output_label)
    result['Gen_label'][index] = parsed_response.output_label
    print(result['Gen_label'][index])

In [ ]:
result['Gen_label'].value_counts()

In [ ]:
# Standardize the Gen_label column values
result['Gen_label'] = result['Gen_label'].str.replace(r'\W', '', regex=True).str.lower()

# Map the standardized labels to the final desired labels
result['Gen_label'] = result['Gen_label'].map({
    'nonsarcasm': 'Non-sarcasm',
    'sarcasm': 'Sarcasm'
})

# Check the updated value counts
print(result['Gen_label'].value_counts())

In [ ]:
# Map string labels to integers
label_mapping = {'Non-sarcasm': 0, 'Sarcasm': 1}
new_result['Gen_label'] = new_result['Gen_label'].map(label_mapping)

# Check the result
print(new_result)

In [ ]:
y_true = list(map(int, new_result['Label']))
y_pred = list(map(int, new_result['Gen_label']))
print(classification_report(y_true, y_pred, digits=6))